### Week 3 Application Assignment

Let’s reconsider the customer reward program dataset. In this exercise, you will complete a predictive modeling task where the target variable is binary.

The dataset also contains a column IndustryType, which is created based on the column Industry in the raw data. Note that Industry has many categories. The analyst who prepared the data chose to combine some categories, which resulted in the column IndustryType. IndustryType has five categories: Department, Discount, Grocery, Restaurants, Specialty. You can create a set of dummy variables based on IndustryType in XLMiner by using the Transform functions.

Part I.

Consider logistic regression models with Reward column as the target variable. Fit the model with two indicator variables, one indicating whether a retailer is a discount store (i.e., IndustryType is Discount), and the other indicating whether a retailer is a grocery store (i.e., IndustryType is Grocery). Report the coefficient estimates in the next three questions. [Hint: After you create the dummy variables, use them as Selected Variables (instead of Categorical Variables) in the first step of Logistic Regression.]

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import datetime

%matplotlib inline
sns.set_style('dark')
sns.set(font_scale=1.2)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from pycaret.classification import *

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

In [2]:
df = pd.read_csv("crp_cleandata3.csv")

In [3]:
df.head()

,Retailer,Salerank,X2013USSales,X2013WorldSales,ProfitMargin,NumStores,Industry,Reward,ProgramName,RewardType,RewardStructure,RewardSize,ExpirationMonth,IndustryType
0,A&P,74,5.831,5.831,48.85,0.277,"Discount, Variety Stores",0,No rewards program,-,-,-,-,Discount
1,Albertsons,21,19.452,19.452,69.02,1.024,Grocery Stores,0,No rewards program,-,-,-,-,Grocery
2,Aldi,38,10.898,10.650,69.41,1.328,Grocery Stores,0,No rewards program,-,-,-,-,Grocery
3,Alimentation Couche Tard (Circle K),82,4.755,8.551,68.03,3.826,Grocery Stores,0,No rewards program,-,-,-,-,Grocery
4,Apple Stores,15,26.648,30.736,11.07,0.254,ElectronicEquipment,0,No rewards program,-,-,-,-,Specialty


In [4]:
df.describe(include='all')

,Retailer,Salerank,X2013USSales,X2013WorldSales,ProfitMargin,NumStores,Industry,Reward,ProgramName,RewardType,RewardStructure,RewardSize,ExpirationMonth,IndustryType
count,100,100.000000,100.000000,100.000000,100.00000,100.000000,100,100.00,100,96,100,100,100,100
unique,100,NaN,NaN,NaN,NaN,NaN,17,NaN,58,11,57,27,11,5
top,Wakefern / Shoprite,NaN,NaN,NaN,NaN,NaN,Grocery Stores,NaN,No rewards program,-,-,-,-,Specialty
freq,1,NaN,NaN,NaN,NaN,NaN,18,NaN,40,37,41,44,44,43
mean,NaN,50.500000,18.373500,24.131540,45.27300,2.698760,NaN,0.55,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,29.011492,36.476003,50.845864,29.23139,3.997641,NaN,0.50,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.000000,3.600000,3.600000,1.02000,0.000000,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,25.750000,5.206750,6.108250,19.44500,0.338500,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,50.500000,8.348500,9.629000,42.02000,1.331500,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,75.250000,16.841000,22.131500,69.57250,3.519750,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN


### Exploratory Data Analysis

In [5]:
df.columns

Index(['Retailer', 'Salerank', 'X2013USSales', 'X2013WorldSales',
       'ProfitMargin', 'NumStores', 'Industry', 'Reward', 'ProgramName',
       'RewardType', 'RewardStructure', 'RewardSize', 'ExpirationMonth',
       'IndustryType'],
      dtype='object')

In [6]:
df.drop(['Retailer','Salerank', 'X2013USSales', 'X2013WorldSales','ProfitMargin', 'NumStores',
         'Industry', 'ProgramName','RewardType', 'RewardStructure', 'RewardSize', 'ExpirationMonth'],axis=1,inplace=True)

In [7]:
df.head()

,Reward,IndustryType
0,0,Discount
1,0,Grocery
2,0,Grocery
3,0,Grocery
4,0,Specialty


In [8]:
df["IndustryType_Discount"] = np.where(df["IndustryType"] == "Discount",1,0)

In [9]:
df["IndustryType_Grocery"] = np.where(df["IndustryType"] == "Grocery",1,0)

In [10]:
df

,Reward,IndustryType,IndustryType_Discount,IndustryType_Grocery
0,0,Discount,1,0
1,0,Grocery,0,1
2,0,Grocery,0,1
3,0,Grocery,0,1
4,0,Specialty,0,0
...,...,...,...,...
95,1,Specialty,0,0
96,1,Specialty,0,0
97,1,Specialty,0,0
98,1,Specialty,0,0


In [11]:
df["Reward"].value_counts()

1    55
0    45
Name: Reward, dtype: int64

### Linear Regression

In [12]:
df.columns

Index(['Reward', 'IndustryType', 'IndustryType_Discount',
       'IndustryType_Grocery'],
      dtype='object')

In [13]:
y = df['Reward']
x1 = df[['IndustryType_Discount', 'IndustryType_Grocery']]

In [14]:
x = sm.add_constant(x1)

In [15]:
results = sm.OLS(y,x).fit()

In [16]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Reward   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     2.101
Date:                Fri, 21 Aug 2020   Prob (F-statistic):              0.128
Time:                        18:36:15   Log-Likelihood:                -69.956
No. Observations:                 100   AIC:                             145.9
Df Residuals:                      97   BIC:                             153.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.6250      0.062     10.111      0.000       0.502       0.748
IndustryType_Discount    -0.2361      0.132     -1.790      0.077      -0.498       0.026
IndustryType_Grocery     -0.1806      0.132     -1.368      0.174      -0.442       0.081
==============================================================================
Omnibus:                     1138.476   Durbin-Watson:                   0.131
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               13.950
Skew:                          -0.200   Prob(JB):                     0.000935
Kurtosis:                       1.214   Cond. No.                         3.16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
df.head()

,Reward,IndustryType,IndustryType_Discount,IndustryType_Grocery
0,0,Discount,1,0
1,0,Grocery,0,1
2,0,Grocery,0,1
3,0,Grocery,0,1
4,0,Specialty,0,0


In [18]:
df2 = df.drop(['IndustryType'],axis=1)

In [19]:
df2.head()

,Reward,IndustryType_Discount,IndustryType_Grocery
0,0,1,0
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,0


In [20]:
X = df2.iloc[:,1:]
y = df2.iloc[:,0]

In [21]:
X.values, y.values

(array([[1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 1],
        [0, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 1],
        [0, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 0],
        [0, 1],
        [0, 1],
        [0, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 1],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        

In [22]:
#train_test_split()

In [23]:
logistic = LogisticRegression()

In [24]:
logistic.fit(X,y)

LogisticRegression()

### Use PyCaret

Part II.

Split the dataset into training and validation sets using a 60:40 split (set the seed for partitioning to 12345; this should be the default value if you have not changed it). [Hint: note that there two Partition buttons in XLMiner ribbon. You should use the Partition->Standard Partition in the Data Mining group.] Report the new coefficient estimates in the next three questions. Use the same two predictor variables as in Part I.

In [25]:
exp = setup(data=df2, train_size=0.6, target = 'Reward', session_id=12345, 
            categorical_features=['IndustryType_Discount','IndustryType_Grocery'])

Setup Succesfully Completed!


,Description,Value
0,session_id,12345
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(100, 3)"
4,Missing Values,False
5,Numeric Features,0
6,Categorical Features,2
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [26]:
lr = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5000,0.3750,0.7500,0.6000,0.6667,-0.2857,-0.3162
1,0.6667,1.0000,1.0000,0.6667,0.8000,0.0000,0.0000
2,0.6667,0.5625,0.7500,0.7500,0.7500,0.2500,0.2500
3,0.6667,0.6667,0.6667,0.6667,0.6667,0.3333,0.3333
4,0.6667,0.6667,1.0000,0.6000,0.7500,0.3333,0.4472
5,0.3333,0.3333,0.6667,0.4000,0.5000,-0.3333,-0.4472
6,0.6667,0.6667,0.6667,0.6667,0.6667,0.3333,0.3333
7,0.6667,0.6667,1.0000,0.6000,0.7500,0.3333,0.4472
8,0.8333,0.8333,1.0000,0.7500,0.8571,0.6667,0.7071
9,0.3333,0.3889,0.3333,0.3333,0.3333,-0.3333,-0.3333


In [27]:
evaluate_model(lr)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…